In [38]:
from copy import deepcopy
import os
import math
import shutil
import re
from collections import defaultdict

import pandas as pd
import rispy

### Issues
- Welke auteursnaam "standard name"? Bibliografisch niet helemaal correct, maar we gaan toch een moeten kiezen. Heeft gevolgen voor toekomstige opname.
- Resoluut verwijderd:
    + Enkel naam auteur blijft behouden (cf. GDPR)
    + Full-text en links daarnaar (want links naar DBNL); externe URLs blijven wél behouden
    + Enkel oorspronkelijke auteur van het record + datum van creatie wordt behouden (aanpasser en datum van aanpassing niet)
- Kan ik in de dump ergens "korte" tijdschriftitels vinden?
- Ik dump nu naar RIS:
    + "Extra" veld op Zotero blijft onbereikbaar (nu naar notes...) bij zowel import als export.
    + Problematische mappings BNTL publicatie-type naar RIS (Lastig, want deze zijn eigenlijk geen attribuut van publicatie, maar van inhoud.):
            'artikelwebsite': 'EJOUR', # suggereert tijdschrift?
            'specialetijdschriftaflevering': 'JFULL', # Journal/periodical (full) -> niet speciaal aan te geven?
            'recensie': 'RPRT', # Report

##### Validering
- Klopt volgorde auteurs?
- Niets vergeten?
- Komen aantallen overeen met website?

In [39]:
dump_dir = '../data/bntl-db-csv-dump-4March2024'

### Authors

In [40]:
authors = pd.read_csv(f"{dump_dir}/publication_authors.csv", header=0)
authors.sample(10)

,publication_id,author_id,sequence
149181,298492,14110,580306
146741,145623,29116,596744
201804,206647,1959,252982
26446,38666,19890,32731
302194,53974,26891,494665
90108,92897,7693,112408
164224,296990,8859,567457
63685,67571,7748,563374
76416,19701,17811,576177
121394,92732,21742,527226


In [41]:
pubid2authorid = defaultdict(list)
for pub_id, author_id in zip(authors['publication_id'], authors['author_id']): # komen auteurs er in de juiste volgorde uit?
    pubid2authorid[pub_id].append(author_id)

In [42]:
authors = pd.read_csv(f"{dump_dir}/authors.csv", header=0)
authors.sample(10)

,id,ppn,is_alias,original_author_id,standardname,fullname,year,summary,creator,creation_date,modifier,modification_date,canonical_url
14211,16799,072273879,f,NaN,"Haar, Bert de","Haar, Bert de",NaN,NaN,1,NaN,NaN,NaN,haar_bert_de
1662,14536,071446575,f,NaN,"Beverwijk, J. M. van","Beverwijk, Johan Maurits van",fl 1809-1823,"fl. 1809-1823, dichter.",1,NaN,NaN,NaN,beverwijk_johan_maurits_van
47655,48779,NaN,t,47425.0,"Vries, Attie de",NaN,NaN,NaN,1,NaN,NaN,NaN,vries_attie_de
5574,40202,170563472,f,NaN,"Bruijne, Mieke de","Bruijne, Mieke de",NaN,NaN,1,NaN,18.0,"2008-04-23, 12:04:11",bruijne_mieke_de
47148,33392,11070665X,f,NaN,"Vochteloo, Marit","Vochteloo, Marit",1966-,NaN,1,NaN,NaN,NaN,vochteloo_marit
17133,66956,NaN,f,NaN,"Banken, Isabel",NaN,NaN,NaN,19,NaN,NaN,NaN,banken_isabel
37650,43432,238817202,f,NaN,"Ros, Myriam","Ros, Myriam",NaN,NaN,1,NaN,NaN,NaN,ros_myriam
22983,68402,NaN,f,NaN,"Berntsen, Bets",NaN,NaN,NaN,16,"2009-03-10, 09:26:50",NaN,NaN,berntsen_bets
33760,11205,070187347,f,NaN,"Parandowski, Jan","Parandowski, Jan",1895-1978,NaN,1,NaN,NaN,NaN,parandowski_jan
49842,34687,120727676,f,NaN,"Winden, Piet van","Winden, Piet van",NaN,NaN,1,NaN,NaN,NaN,winden_piet_van


In [43]:
authorid2author = dict(zip(authors['id'], authors['standardname']))

### Keywords

In [44]:
publ_keywords = pd.read_csv(f"{dump_dir}/publications_keywords.csv", header=0)
publ_keywords.sample(10)

,publication_id,keyword_id,sequence
539238,268176,652,724160
599079,221610,14676,849353
144771,171400,17107,1304301
624525,5361,47054,893171
452895,167984,47266,616499
234006,136114,47174,317525
5944,259732,16690,8088
195228,112318,46965,264289
595777,209907,5730,842762
318184,200999,1819,432486


In [45]:
pubid2keywordid = defaultdict(list)
for pub_id, keyword_id in zip(publ_keywords['publication_id'], publ_keywords['keyword_id']):
    pubid2keywordid[pub_id].append(keyword_id)

In [46]:
keywords = pd.read_csv(f"{dump_dir}/keywords.csv", header=0)
keywords.sample(10)

,id,headword,summary,variants,author_id,creator,type,ppn,kmc260,kmc265,creation_date,modifier,modification_date,canonical_url
7843,39873,De steen die trect die naelde naer,"lied, nr. 95 van het Gruuthuse-hs.",; Steen die trect die naelde naer,NaN,1,zaak,159028787,NaN,Steen die trect die naelde naer,NaN,NaN,NaN,de_steen_die_trect_die_naelde_naer
20457,40863,literaire voorkeur van musici,NaN,voorkeur van musici (literaire),NaN,1,zaak,16909877X,voorkeur van musici (literaire),NaN,NaN,NaN,NaN,literaire_voorkeur_van_musici
12349,27700,gesproken boeken,NaN,NaN,NaN,1,zaak,079099971,NaN,NaN,NaN,NaN,NaN,gesproken_boeken
23914,32418,NCR Literair prijs,voortgezet als de AT&T Literair-prijs.,; AT&T Literar-prijs,NaN,1,zaak,10260634X,NaN,AT&T Literar-prijs,NaN,NaN,NaN,ncr_literair_prijs
45965,19459,Nederduitsche Letteroefeningen (tijdschr.),1833-1834,NaN,NaN,1,zaak,076726835,NaN,NaN,NaN,19.0,"2014-06-05, 13:09:29",nederduitsche_letteroefeningen_tijdschr
11111,6408,fis (= bunzing),NaN,NaN,NaN,1,zaak,076580474,NaN,NaN,NaN,NaN,NaN,fis_bunzing
20490,28158,literatuur en fotografie,NaN,NaN,NaN,1,zaak,080395910,NaN,NaN,NaN,NaN,NaN,literatuur_en_fotografie
28602,33418,receptie van John Bunyan,NaN,NaN,NaN,1,zaak,108603334,NaN,NaN,NaN,NaN,NaN,receptie_van_john_bunyan
28679,21351,reconstructie,NaN,NaN,NaN,1,zaak,07674695X,NaN,NaN,NaN,NaN,NaN,reconstructie
6619,34076,Concordia (leesgezelschap; Veenendaal),1861-1945,NaN,NaN,1,zaak,111835453,NaN,NaN,NaN,NaN,NaN,concordia_leesgezelschap_veenendaal


In [47]:
keywordid2keyword = {}

for kw_id, headword, auth_id, type_ in zip(keywords['id'], keywords['headword'], keywords['author_id'], keywords['type']):
    if type_ == 'auteur':
        keywordid2keyword[kw_id] = authorid2author[auth_id] + ' (auteur)'
    else: 
        keywordid2keyword[kw_id] = headword

### Publications

In [48]:
import numpy as np
publications = pd.read_csv(f"{dump_dir}/publications.csv", header=0, parse_dates=['creation_date', 'modification_date'])

In [49]:
publications['id'] = publications['id'].fillna('')
publications = publications.set_index('id')

In [50]:
author_names, keyword_names = [], []

for pub_id in publications.index:
    ans, kns = [], []

    try:
        for author_id in pubid2authorid[pub_id]:
            ans.append(authorid2author[author_id])
    except KeyError:
        pass

    try:
        for keyword_id in pubid2keywordid[pub_id]:
            kns.append(keywordid2keyword[keyword_id])
    except KeyError:
        pass

    author_names.append(ans)
    keyword_names.append(kns)

publications['authors'] = author_names
publications['keywords'] = keyword_names

In [51]:
# _ in new column names indicates: mapping not available
bntl2ris = {
            'id': 'id',
            'ppn': 'ppn_',
            'type': 'type_of_reference',
            'jaar': 'year',
            'title_description': 'title',
            'abstract': 'abstract', # geen echt abstract, maar meer informatie
            'url': 'urls',
            'containing_publication_id': 'containing_publication_id', # id van het moederrecord
            'containing_publication_ppn': '?',
            'page_numbers': 'page_numbers',
            'creator': 'creator_', # degene die record heeft aangemaakt: gaat verloren
            'creation_date': 'date',
            'modifier': '?', # degend die record het laatste heeft aangepast: gaat verloren
            'modification_date': 'modification_date_', # wanneer record het laatste is aangepast: gaat verloren
            'canonical_url': 'canonical_url_', # deellink naar record op BNTL-sire: gaat verloren
          }

publications = publications.rename(mapper=bntl2ris, axis=1)

In [52]:
# for dev purposes:
clean_publications = publications[[c for c in publications.columns if not (c.endswith('_') or c == '?')]].copy()
#clean_publications = clean_publications.sample(1000)
clean_publications

,type_of_reference,year,title,abstract,urls,containing_publication_id,page_numbers,date,authors,keywords
id,,,,,,,,,,
254444,artikeltijdschrift,2009,"Campenhout, Frans van. Ropsiennes, de nieuwe dichtbundel van Mark Meekers.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 14.","Over: Meekers, Mark. Ropsiennes: gedichten bij het leven en werk van Félicien Rops (1833-1898). Leuven: P, 2009.",NaN,NaN,NaN,2009-06-26 11:32:26,"[Campenhout, Frans van]","[Meekers, Mark (auteur)]"
254442,artikeltijdschrift,2009,"Campenhout, Frans van. Dichteres en romancière Liane Bruylants overleden.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 5-8.",NaN,NaN,NaN,NaN,2009-06-26 11:27:36,"[Campenhout, Frans van]","[Bruylants, Liane (auteur)]"
313784,artikeltijdschrift,2018,"Lievois, Katrien, Hanne Kloots & Noureddine Nahed. Kif-kif?: Arabische ontleningen in de Nederlandse, Spaanse en Arabische versie van ""Kiffe kiffe demain"".\r\nIn: Meertaligheid. Speciaal gedeelte van: Filter (Nijmegen): 25 (2018) 3 (sep) 37-47.","P. 46 Noten; p. 46 Bibliografie; p. 47 Bijlage: vertalingen van ""Kiffe kiffe demain"".",NaN,NaN,NaN,2023-11-29 16:36:19,"[Lievois, Katrien, Kloots, Hanne, Nahed, Noureddine]","[literair vertalen, vertalingen uit het Frans, meertaligheid, multiculturaliteit, Arabische leenwoorden, straattaal]"
171436,artikeltijdschrift,1965,"Interprovinciale Prijs voor monografie aan een Westvlaming, [door] A.S.\r\nIn: West-Vlaanderen: 14 (1965) 79 (jan-feb) 66-67.","N.a.v.: Deblaere, Albert. De mystieke schrijfster Maria Petyt, (1623-1677). Gent: Secretarie der Academie, 1962.",NaN,166925.0,"vol. 14 (1965), afl. 79 (jan-feb), p. 66-67.",NaT,[n.v.t.],"[Deblaere, Albert (auteur), Petyt, Maria (auteur)]"
246627,artikelboek,2004,"Willaert, Frank. Margaret's booklets: memory in ""Vanden seven sloten"" by Jan van Ruusbroec.\r\nIn: Medieval memory: image and text; edited by Frank Willaert, Herman Braet, Tom Mertens, Theofiel Venckeleer. Turnhout: Brepols, 2004, p. 99-128.\r\n(Textes et études du Moyen Âge; 27).","Lezing, gehouden op het congres ""Memory in the Middle Ages"" aan de Universiteit Antwerpen, 7-9 maart 2002.",NaN,NaN,NaN,2008-06-18 12:05:19,"[Willaert, Frank]","[Ruusbroec, Jan van (auteur), memoria, tekstinterpretatie]"
...,...,...,...,...,...,...,...,...,...,...
254432,recensie,1962,"Plard, Henri. Une mystique flamande retrouvée: Maria Petyt (1623-1677).\r\nIn: Études germaniques (Paris): 17 (1962) 3 (juill-sep) 347-354.",NaN,NaN,NaN,NaN,2009-06-26 10:31:29,"[Plard, Henri]",[]
129705,recensie,2003,"Offermans, Cyrille. \r\nIn: Ons erfdeel (Rekkem): 46 (2003) 3 (juni) 446-448.",NaN,NaN,NaN,NaN,NaT,"[Offermans, Cyrille]",[]
129707,boek,2003,"Nederlands-Afghaanse grammatica = Nīdarlandī-Paśtū paśwayah / Sayed Nasir Ahmad (Sayyid Naṣīr Aḥmad). - 3e, herz. dr. - Utrecht : Nederlands Centrum Buitenlanders, cop. 2003. - XIV, 182 p. ; 24 cm Tekst in het Nederlands en het Pasjtoe. - Ook o.d.t.: Nederlands-Pashto grammatica. - Met lit. opg. ISBN 90-5517-169-7.",P. 181-182 Literatuur.\n1e-2e dr. ?,NaN,NaN,NaN,NaT,"[Nasir Ahmad, Sayed]","[leerboeken voor buitenlanders, Afghaanssprekenden, grammatica]"


In [53]:
clean_publications['type_of_reference'].unique()

array(['artikeltijdschrift', 'artikelboek', 'boek', 'recensie',
       'tijdschriftofreeks', 'website', 'specialetijdschriftaflevering',
       'cddvd', 'artikelwebsite'], dtype=object)

In [54]:
# Naar: https://en.wikipedia.org/wiki/RIS_(file_format)#Type_of_reference
map_type = {'artikeltijdschrift': 'JOUR', # Journal/periodical (article)
            'recensie': 'RPRT', # Report
            'artikelboek': 'CHAP', # Book section/chapter
            'boek': 'BOOK', # Book (whole)
            'website': 'WEB', # Web page
            'artikelwebsite': 'EJOUR', # suggereert tijdschrift?
            'specialetijdschriftaflevering': 'JFULL', # Journal/periodical (full) -> niet speciaal aan te geven?
            'tijdschriftofreeks': 'JFULL', # Journal/periodical (full)
            'cddvd': 'EBOOK', # Electronic book
           }

In [55]:
clean_publications['type_of_reference'] = clean_publications['type_of_reference'].replace(map_type)

clean_publications['abstract'] = clean_publications['abstract'].fillna('').str.strip()

clean_publications['start_page'] = clean_publications['page_numbers'].fillna('').str.strip()

clean_publications['date'] = clean_publications['date'].dt.strftime('%d-%m-%Y').fillna('').str.strip()

clean_publications['urls'] = clean_publications['urls'].fillna('').str.strip()
clean_publications['urls'] = [[u] if u else '' for u in clean_publications['urls']]

# same for all
clean_publications['name_of_database'] = 'Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL)'

### Mother references

In [56]:
pd.set_option('display.max_colwidth', None)

In [57]:
secondary_titles = []

for idx, row in clean_publications.iterrows():
    if not math.isnan(row['containing_publication_id']):
        jtitle = publications.loc[int(row['containing_publication_id'])]['title']
        secondary_titles.append(jtitle)
    else:
        secondary_titles.append('')

clean_publications['secondary_title'] = secondary_titles

In [58]:
clean_publications = clean_publications[[c for c in clean_publications.columns \
                                         if c not in ('containing_publication_id', 'page_numbers')]].copy()

Remove literary authors mentioned as publication authors:

In [59]:
clean_authors = []

for authors in clean_publications['authors']:
    if len(authors) == 1 and authors[0].lower() == 'n.v.t.':
        clean_authors.append([])
    else:
        clean_authors.append(authors)

clean_publications['authors'] = clean_authors

In [60]:
clean_authors = []

for authors, keywords in zip(clean_publications['authors'], clean_publications['keywords']):
    new_authors = authors.copy()
    if authors and keywords:
        authors_mentioned = set()
        for kw in keywords:
            if '(auteur)' in kw:
                authors_mentioned.add(kw.strip().replace(' (auteur)', '').strip())
        new_authors = [a for a in authors if a in set(authors) - authors_mentioned]
    clean_authors.append(new_authors)

clean_publications['authors'] = clean_authors


In [61]:
clean_publications

,type_of_reference,year,title,abstract,urls,date,authors,keywords,start_page,name_of_database,secondary_title
id,,,,,,,,,,,
254444,JOUR,2009,"Campenhout, Frans van. Ropsiennes, de nieuwe dichtbundel van Mark Meekers.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 14.","Over: Meekers, Mark. Ropsiennes: gedichten bij het leven en werk van Félicien Rops (1833-1898). Leuven: P, 2009.",,26-06-2009,"[Campenhout, Frans van]","[Meekers, Mark (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
254442,JOUR,2009,"Campenhout, Frans van. Dichteres en romancière Liane Bruylants overleden.\r\nIn: Verba (Brussel): 13 (2009) 2 (mei-juli) 5-8.",,,26-06-2009,"[Campenhout, Frans van]","[Bruylants, Liane (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
313784,JOUR,2018,"Lievois, Katrien, Hanne Kloots & Noureddine Nahed. Kif-kif?: Arabische ontleningen in de Nederlandse, Spaanse en Arabische versie van ""Kiffe kiffe demain"".\r\nIn: Meertaligheid. Speciaal gedeelte van: Filter (Nijmegen): 25 (2018) 3 (sep) 37-47.","P. 46 Noten; p. 46 Bibliografie; p. 47 Bijlage: vertalingen van ""Kiffe kiffe demain"".",,29-11-2023,"[Lievois, Katrien, Kloots, Hanne, Nahed, Noureddine]","[literair vertalen, vertalingen uit het Frans, meertaligheid, multiculturaliteit, Arabische leenwoorden, straattaal]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
171436,JOUR,1965,"Interprovinciale Prijs voor monografie aan een Westvlaming, [door] A.S.\r\nIn: West-Vlaanderen: 14 (1965) 79 (jan-feb) 66-67.","N.a.v.: Deblaere, Albert. De mystieke schrijfster Maria Petyt, (1623-1677). Gent: Secretarie der Academie, 1962.",,,[],"[Deblaere, Albert (auteur), Petyt, Maria (auteur)]","vol. 14 (1965), afl. 79 (jan-feb), p. 66-67.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),"West-Vlaanderen: tweemaandelijks tijdschrift voor kunst en cultuur; uitgegeven door het Christelijk Vlaams Kunstenaars Verbond. St. Michiels-Brugge: Christelijk Vlaams Kunstenaarsverbond, 1950-1965."
246627,CHAP,2004,"Willaert, Frank. Margaret's booklets: memory in ""Vanden seven sloten"" by Jan van Ruusbroec.\r\nIn: Medieval memory: image and text; edited by Frank Willaert, Herman Braet, Tom Mertens, Theofiel Venckeleer. Turnhout: Brepols, 2004, p. 99-128.\r\n(Textes et études du Moyen Âge; 27).","Lezing, gehouden op het congres ""Memory in the Middle Ages"" aan de Universiteit Antwerpen, 7-9 maart 2002.",,18-06-2008,"[Willaert, Frank]","[Ruusbroec, Jan van (auteur), memoria, tekstinterpretatie]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
...,...,...,...,...,...,...,...,...,...,...,...
254432,RPRT,1962,"Plard, Henri. Une mystique flamande retrouvée: Maria Petyt (1623-1677).\r\nIn: Études germaniques (Paris): 17 (1962) 3 (juill-sep) 347-354.",,,26-06-2009,"[Plard, Henri]",[],,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
129705,RPRT,2003,"Offermans, Cyrille. \r\nIn: Ons erfdeel (Rekkem): 46 (2003) 3 (juni) 446-448.",,,,"[Offermans, Cyrille]",[],,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),
129707,BOOK,2003,"Nederlands-Afghaanse grammatica = Nīdarlandī-Paśtū paśwayah / Sayed Nasir Ahmad (Sayyid Naṣīr Aḥmad). - 3e, herz. dr. - Utrecht : Nederlands Centrum Buitenlanders, cop. 2003. - XIV, 182 p. ; 24 cm Tekst in het Nederlands en het Pasjtoe. - Ook o.d.t.: Nederlands-Pashto grammatica. - Met lit. opg. ISBN 90-5517-169-7.",P. 181-182 Literatuur.\n1e-2e dr. ?,,,"[Nasir Ahmad, Sayed]","[leerboeken voor buitenlanders, Afghaanssprekenden, grammatica]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),


## Dump to RIS

In [62]:
ris_mapping = deepcopy(rispy.TAG_KEY_MAPPING)
ris_mapping['DB'] = 'name_of_database'

In [63]:
def dump2ris(pubs, bn):
    records = []
    for record in pubs.to_dict(orient='records'):
        records.append({k:v for k, v in record.items() if v})

    with open(f'{bn}.ris', 'w') as bibliography_file:
        rispy.dump(records, bibliography_file, mapping=ris_mapping)
    

In [64]:
def clean_dir(dir_name):
    try:
        shutil.rmtree(dir_name)
    except FileNotFoundError:
        pass
    os.mkdir(dir_name)

dump_dir = '../data/ris-dump'
clean_dir(dump_dir)

In [65]:
def to_decade(year):
    try:
        decade = int(str(int(year))[:-1])
        if decade < 194: # lump everything pre-1945 together under misc
            decade = 'misc'
        else:
            decade = str(decade) + '0s'
    except ValueError:
        decade = 'misc'
    
    return decade

clean_publications.loc[:, 'decade'] = clean_publications['year'].apply(to_decade).copy()
clean_publications.value_counts('decade')

decade
1990s    67537
1980s    54628
2000s    46647
1970s    36057
2010s    31339
1960s    25846
1940s    13171
misc     11223
1950s     8313
2020s     8104
2030s        1
Name: count, dtype: int64

In [66]:
clean_publications[clean_publications['secondary_title'].str.contains('Awater')]

,type_of_reference,year,title,abstract,urls,date,authors,keywords,start_page,name_of_database,secondary_title,decade
id,,,,,,,,,,,,
143486,JOUR,2004,"Wat bezielt : [interview met Bart F.M. Droog] / door Thomas Möhlmann.\nIn: Awater Rotterdam: vol. 3 (2004), afl. 3 (najaar), pag. 25-27.",,,,"[Droog, Bart F. M., Möhlmann, Thomas]","[literatuurwetenschap: per auteur - vanaf 1830 - descriptieve studies, biografische bronnen: interviews, vraaggesprekken, Droog, Bart]","vol. 3 (2004), afl. 3 (najaar), p. 25-27.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),"Awater; uitgave van Stichting PoëzieClub. Rotterdam: Stichting PoëzieClub, 2002-....",2000s
143524,JFULL,2004,"Het grote Gerrit Komrij themanummer / [red.: Onno Blom ... et al.; met bijdragen van: Onno Blom ... et al.]. - Rotterdam : Stichting PoëzieClub, 2004. - 24 p. : ill. ; 42 cm. - (Awater, ISSN 1570-0917 ; jrg. 3 (2004) nr. 1) ISBN 90-453-0216-0.",,,,"[Blom, Onno]","[literatuurwetenschap: per auteur - vanaf 1830 - descriptieve studies, algemeen biografisch: leven en werk algemeen, Komrij, Gerrit (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),"Awater; uitgave van Stichting PoëzieClub. Rotterdam: Stichting PoëzieClub, 2002-....",2000s
149628,JOUR,2005,"De ware volkspoëzie : bij poetry slams verliest de poëzie het van effectbejag / door Erik Jan Harmens.\nIn: Awater Rotterdam: vol. 4 (2005), afl. 2 (zomer), pag. 16-19.",,,,"[Harmens, Erik Jan]","[literatuurwetenschap: vanaf 1830 - afzonderlijke onderwerpen en problemen van algemene aard - descriptieve studies, podiumdichters, poëzie]","vol. 4 (2005), afl. 2 (zomer), p. 16-19.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),"Awater; uitgave van Stichting PoëzieClub. Rotterdam: Stichting PoëzieClub, 2002-....",2000s
149621,JOUR,2005,"Blondeau, Thomas. Een onbestemde geilheid: interview met dichter Piet Gerbrandy.\r\nIn: Awater (Amsterdam): 4 (2005) 2 (zomer) 4-7.",,,,"[Blondeau, Thomas]","[Gerbrandy, Piet (auteur)]","vol. 4 (2005), afl. 2 (zomer), p. 4-7.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),"Awater; uitgave van Stichting PoëzieClub. Rotterdam: Stichting PoëzieClub, 2002-....",2000s
149677,JOUR,2005,"Schouten, Rob. Genezen van mensenhaat.\r\nIn: Awater (Rotterdam): 4 (2005) 1 (winter) 32-33.","Over: Sleutelaar, Hans. Vermiste stad: Rotterdamse kwatrijnen. Amsterdam: De Bezige Bij, 2004.",,,"[Schouten, Rob]","[Sleutelaar, Hans (auteur)]","vol. 4 (2005), afl. 1 (wi!074002953!nter), p. 32-33.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),"Awater; uitgave van Stichting PoëzieClub. Rotterdam: Stichting PoëzieClub, 2002-....",2000s
...,...,...,...,...,...,...,...,...,...,...,...,...
268991,JOUR,2011,"Veelen, Arjen van. Gebruik de taal met optimale wellust.\r\nIn: Awater (Rotterdam): 10 (2011) 1 (winter) 4-8.","Interview naar aanleiding van: Brassinga, Anneke. Ontij: gedichten. Amsterdam: De Bezige Bij, 2010.",,09-05-2011,"[Veelen, Arjen van]","[Brassinga, Anneke (auteur)]",,Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),"Awater; uitgave van Stichting PoëzieClub. Rotterdam: Stichting PoëzieClub, 2002-....",2010s
149767,JOUR,2007,"Hoorne, Philip. Knar met potentieel.\r\nIn: Awater (Rotterdam): 6 (2007) 1 (winter) 38-39.","Over: Lehmann, L.Th. Wat boven kwam: gedichten. Amsterdam, 2006.",,,"[Hoorne, Philip]","[Lehmann, L. Th. (auteur)]","vol. 6 (2007), afl. 1 (winter), p. 38-39.",Bibliografie van de Nederlandse Taal- en Literatuurwetenschap (BNTL),"Awater; uitgave van Stichting PoëzieClub. Rotterdam: Stichting PoëzieClub, 2002-....",2000s
149703,JOUR,2006,"Meisjes bezingen / door Thomas Möhlmann.\nIn: Awater Rotterdam: vol. 5 (2006), afl. 1 (winter), pag. 38-39.","Over: Woudsma, Co. Geluksinstructies: gedichten. Amsterdam, 2005.",,,"[Möhlmann, Thomas]","[literatuurwetenschap: per auteur - vanaf 1830 - descriptieve studies, besprekingsartikelen en recensies over één werk, Woudsma, C

In [67]:
sizes = []
for decade, decade_pubs in clean_publications.groupby('decade'):
    clean_dir(f'{dump_dir}/{decade}')
    print(f'- {decade}')
    for ptype, ptype_pubs in decade_pubs.groupby('type_of_reference'):
        print('    +', ptype, len(ptype_pubs))
        dump2ris(ptype_pubs.drop('decade', axis=1), f'{dump_dir}/{decade}/{ptype}')
        sizes.append((len(ptype_pubs), decade, ptype))

sizes.sort(reverse=True)
print(sizes[:5])

- 1940s
    + BOOK 1442
    + CHAP 1764
    + JFULL 68
    + JOUR 7739
    + RPRT 2158
- 1950s
    + BOOK 917
    + CHAP 1161
    + JFULL 17
    + JOUR 4414
    + RPRT 1804
- 1960s
    + BOOK 2185
    + CHAP 4084
    + JFULL 190
    + JOUR 16004
    + RPRT 3383
- 1970s
    + BOOK 3623
    + CHAP 6650
    + JFULL 273
    + JOUR 21101
    + RPRT 4410
- 1980s
    + BOOK 6722
    + CHAP 12289
    + EBOOK 2
    + JFULL 619
    + JOUR 28668
    + RPRT 6327
    + WEB 1
- 1990s
    + BOOK 7992
    + CHAP 14247
    + EBOOK 33
    + EJOUR 97
    + JFULL 660
    + JOUR 33114
    + RPRT 11384
    + WEB 10
- 2000s
    + BOOK 6229
    + CHAP 11027
    + EBOOK 54
    + EJOUR 628
    + JFULL 479
    + JOUR 21238
    + RPRT 6507
    + WEB 485
- 2010s
    + BOOK 3891
    + CHAP 6300
    + EBOOK 9
    + EJOUR 569
    + JFULL 442
    + JOUR 15117
    + RPRT 4926
    + WEB 85
- 2020s
    + BOOK 824
    + CHAP 1627
    + EBOOK 2
    + EJOUR 251
    + JFULL 161
    + JOUR 4196
    + RPRT 1042
    + WEB 1
- 2

### Journal titles

Dump table of original journal title, normalized journal title and ISSN for manual correction so that it can be re-used for mapping:

In [75]:
journal_pubs = clean_publications[clean_publications['type_of_reference'].isin({'JOUR', 'JFULL'})].copy()
# some pubs lack a secondary journal title...
journal_pubs = journal_pubs[~journal_pubs['secondary_title'].isna()]
journal_pubs = journal_pubs[journal_pubs['secondary_title'].str.strip() != '']

In [76]:
title_splitter = re.compile(r'(?<=([a-z0-9]))(\.\-*\s)|(\s\/|\;\s)')
def clean_journal_title(title):
    title = title.replace('...', '')
    clean_title = title_splitter.split(title)[0]
    clean_title = clean_title.replace(' : ', ': ').strip()
    clean_title = clean_title.split(': ')[0].strip()
    return clean_title

In [77]:
jour = journal_pubs['secondary_title'].value_counts().reset_index()
jour['normalized'] = jour['secondary_title'].apply(clean_journal_title)
jour = jour[['secondary_title', 'normalized', 'count']]

In [78]:
jour.head(30)

,secondary_title,normalized,count
0,"Ons erfdeel: kultureel tijdschrift voor Zuidvlaamse werking in Vlaams-Nederlands-Suidafrikaanse samenwerking. Menen: Deleu, 1957-...",Ons erfdeel,2660
1,"Dietsche warande en Belfort: tijdschrift voor letterkunde, kunst en geestesleven. Gent: Siffer; Haarlem: Coebergh; Antwerpen [etc.]: Standaard, 1900-....",Dietsche warande en Belfort,2461
2,"De nieuwe taalgids: tweemaandelijks tijdschrift. Groningen: Tjeenk Willink, 1907-1995.",De nieuwe taalgids,2359
3,"Bzzlletin; Stichting BZZTôH Teater. Voorburg: Stichting BZZTôH, 1972-2004.",Bzzlletin,1638
4,"Poëziekrant: tweemaandelijks tijdschrift. Gent: Poëziecentrum, 1976-...",Poëziekrant,1573
5,"Onze taal: maandblad van het Genootschap Onze Taal. Amsterdam: Genootschap Onze Taal, 1932-....",Onze taal,1322
6,"Vlaanderen: tweemaandelijks tijdschrift voor kunst en cultuur; Christelijk Vlaams Kunstenaarsverbond. Roeselare: Christelijk Vlaams Kunstenaarsverbond, 1966-2017.",Vlaanderen,1312
7,"De gids: nieuwe vaderlandsche letteroefeningen. Amsterdam: G.J.A. Beijerinck [etc.], 1837-....",De gids,1249
8,"Levende talen: berichten en mededelingen van de Vereniging van Leraren in Levende Talen. Utrecht: Vereniging van Leraren in Levende Talen, 1930-1999.",Levende talen,1239
9,"Tijdschrift voor Nederlandse taal- en letterkunde, TNTL; uitgegeven vanwege de Maatschappij der Nederlandsche Letterkunde te Leiden. Leiden: Brill, 1881-....","Tijdschrift voor Nederlandse taal- en letterkunde, TNTL",962


In [79]:
fjour = clean_publications[clean_publications['type_of_reference'] == 'JFULL']
fjour = fjour[fjour['authors'].str.len() == 0] # avoid special issues etc.

issn_pattern = re.compile(r'ISSN (\d{4}-\d{3}[\dX])')

def extract_issn(text):
    issns = list(re.findall(issn_pattern, text))
    if issns:
        return issns[0] # select first, if multiple mentioned
    else:
        return None
    
fjour['issn'] = fjour['abstract'].apply(extract_issn)
fjour['normalized'] = fjour['title'].apply(clean_journal_title)
fjour = fjour[['normalized', 'issn']]
fjour = fjour[~fjour['issn'].isna()]
fjour

,normalized,issn
id,,
129758,Jaarboek Jet Jorssen Genootschap,1378-0883
129767,Cultuurbericht,1872-1265
139578,Villers,1371-6360
161368,De bibliotheekgids,0006-1956
164819,Karakter,0022-8982
...,...,...
126608,Jaarboek,0770-7517
126723,Sjeppelroot thir(s)ty one,0772-8921
126904,Lyra,1570-4904


In [80]:
journal2issn = dict(zip(fjour['normalized'], fjour['issn']))

jour['issn'] = [journal2issn[nt]  if nt in journal2issn else ''\
                for nt in jour['normalized']]
jour

,secondary_title,normalized,count,issn
0,"Ons erfdeel: kultureel tijdschrift voor Zuidvlaamse werking in Vlaams-Nederlands-Suidafrikaanse samenwerking. Menen: Deleu, 1957-...",Ons erfdeel,2660,0030-2651
1,"Dietsche warande en Belfort: tijdschrift voor letterkunde, kunst en geestesleven. Gent: Siffer; Haarlem: Coebergh; Antwerpen [etc.]: Standaard, 1900-....",Dietsche warande en Belfort,2461,0012-2645
2,"De nieuwe taalgids: tweemaandelijks tijdschrift. Groningen: Tjeenk Willink, 1907-1995.",De nieuwe taalgids,2359,0028-9922
3,"Bzzlletin; Stichting BZZTôH Teater. Voorburg: Stichting BZZTôH, 1972-2004.",Bzzlletin,1638,0165-0858
4,"Poëziekrant: tweemaandelijks tijdschrift. Gent: Poëziecentrum, 1976-...",Poëziekrant,1573,2030-0638
...,...,...,...,...
4202,"Strippamflet. - ([1975]) [1] - ([1975]) 5. - Apeldoorn : De Jonge, [1975]. - ? cm Verschijnt 5x per jaar. - Niet verder verschenen. ([1975])",Strippamflet,1,
4203,"De schalm : tijdschrift voor buurt- en buurthuiswerk / uitg. van de Nederlandse Bond van Volkshuizen. - Jrg. 1 (1935/36) - jrg. 27, nr. 6 (1970). - Utrecht : Nederlandse Bond van Volkshuizen [etc.], 1935-1970. - 19-23 cm Verscheen 1x per 2 maanden. - Ondertitel varieert. - Corporatie later o.d.n.: Nederlandse Bond voor Sociaal-Cultureel Vormingswerk. - Als interim-nummer verscheen in aug. 1971: Missing link. - Voortgez. als: Attak = ISSN 0044-9962. - Met: De Schalm bibliografie, 1935-1970, 27 jaargangen. - 1972.",De schalm,1,
4204,"27 Squibs; [redactie: G. E. Booij... et al.]. Groningen: Wolters-Noordhoff, 1977. 116 p. (p. 377-492).\r\n(Speciaal nummer van: Spektator; jrg. 6, nr. 7-8, (feb-mrt)).",27 Squibs,1,
4205,"The journal of ecclesiastical history. London: Faber and Faber, 1950-....",The journal of ecclesiastical history,1,0022-0469


In [81]:
jour.to_excel('../data/journal_titles.xlsx', header=True, index=False)